## CAE を使ってノイズ除去処理

In [1]:
import os
import glob
import math
import random
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model,Sequential
from tensorflow.python.keras.layers import Conv2D,Dense,Input,MaxPooling2D, UpSampling2D,Lambda
from tensorflow.python.keras.preprocessing.image import load_img,img_to_array,array_to_img,ImageDataGenerator

### data input

In [2]:
from tensorflow.python.keras.datasets import mnist
(x_train,_),(x_test,_) = mnist.load_data()
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)
x_train = x_train / 255
x_test = x_test / 255

### add noise data
###### (Masking Noise: modify parts of the data to 0 /  Gaussian Noise: Gaussian random data

In [3]:
def make_masking_noise_data(data_x, percent = 0.1):
    size = data_x.shape
    masking = np.random.binomial(n=1, p=percent, size = size)
    return data_x * masking
x_train_masked = make_masking_noise_data(x_train)
x_test_masked = make_masking_noise_data(x_test)

In [4]:
def make_gaussian_noise_data(data_x,scale=0.8):
    gaussian_data_x = data_x + np.random.normal(loc = 0, scale=scale, size=data_x.shape)
    gaussian_data_x = np.clip(gaussian_data_x,0,1)
    return gaussian_data_x
x_train_gauss = make_gaussian_noise_data(x_train)
x_test_gauss = make_gaussian_noise_data(x_test)

### image compare: original, masked, gaussian

In [5]:
from IPython.display import display_png
display_png(array_to_img(x_train[0]))
display_png(array_to_img(x_train_gauss[0]))
display_png(array_to_img(x_train_masked[0]))

### Create CAE model

In [8]:
autoencoder = Sequential()
#Encoder
autoencoder.add(
    Conv2D(
        16,(3,3),1,
        activation= 'relu',
        padding = 'same',
        input_shape=(28,28,1)
    )
)
autoencoder.add(
    MaxPooling2D(
        (2,2),
        padding='same'
    )
)
autoencoder.add(
    Conv2D(
        8,(3,3),1,
        activation = 'relu',
        padding = 'same'
    )
)
autoencoder.add(
    MaxPooling2D(
        (2,2),
        padding='same'
    )
)

#Decoder
autoencoder.add(
    Conv2D(
        8,(3,3),1,
        activation = 'relu',
        padding = 'same'
    )
)
autoencoder.add(UpSampling2D((2,2)))
autoencoder.add(
    Conv2D(
        16,(3,3),1,
        activation = 'relu',
        padding = 'same'
    )
)
autoencoder.add(UpSampling2D((2,2)))
autoencoder.add(
    Conv2D(
        1,(3,3,),1,
        activation = 'sigmoid',
        padding = 'same'
    )
)

autoencoder.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy'
)
initial_weights = autoencoder.get_weights()

In [9]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 8)           584       
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 16)        1168      
__________

### input gauss noise data and compare with the original data and begin training

In [21]:
autoencoder.set_weights(initial_weights)
autoencoder.fit(
    x_train_gauss,
    x_train,
    epochs = 10,
    batch_size = 20,
    shuffle = True
)

Epoch 1/10
60000/60000 [==============================] - 19s 319us/step - loss: 0.1780
Epoch 2/10
60000/60000 [==============================] - 18s 299us/step - loss: 0.1576
Epoch 3/10
60000/60000 [==============================] - 18s 299us/step - loss: 0.1523
Epoch 4/10
60000/60000 [==============================] - 18s 299us/step - loss: 0.1495
Epoch 5/10
60000/60000 [==============================] - 18s 300us/step - loss: 0.1477
Epoch 6/10
60000/60000 [==============================] - 18s 300us/step - loss: 0.1468
Epoch 7/10
60000/60000 [==============================] - 18s 300us/step - loss: 0.1461
Epoch 8/10
60000/60000 [==============================] - 18s 303us/step - loss: 0.1455
Epoch 9/10
60000/60000 [==============================] - 19s 308us/step - loss: 0.1449
Epoch 10/10
60000/60000 [==============================] - 19s 311us/step - loss: 0.1444


### predict output data using the gauss noise model

In [26]:
gauss_preds = autoencoder.predict(x_test_gauss)

### output the result of image 

In [27]:
for i in range(10):
    display_png(array_to_img(x_test[i]))
    display_png(array_to_img(x_test_gauss[i]))
    display_png(array_to_img(gauss_preds[i]))
    print('-'*25)

-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


### reset weights and train by masking noise data

In [17]:
autoencoder.set_weights(initial_weights)
autoencoder.fit(
    x_train_masked,
    x_train,
    epochs = 10,
    batch_size = 20,
    shuffle = True
)

Epoch 1/10
60000/60000 [==============================] - 19s 314us/step - loss: 0.2038
Epoch 2/10
60000/60000 [==============================] - 19s 312us/step - loss: 0.1886
Epoch 3/10
60000/60000 [==============================] - 19s 312us/step - loss: 0.1855
Epoch 4/10
60000/60000 [==============================] - 19s 311us/step - loss: 0.1840
Epoch 5/10
60000/60000 [==============================] - 19s 319us/step - loss: 0.1830
Epoch 6/10
60000/60000 [==============================] - 19s 316us/step - loss: 0.1822
Epoch 7/10
60000/60000 [==============================] - 19s 316us/step - loss: 0.1815
Epoch 8/10
60000/60000 [==============================] - 19s 314us/step - loss: 0.1810
Epoch 9/10
60000/60000 [==============================] - 19s 311us/step - loss: 0.1805
Epoch 10/10
60000/60000 [==============================] - 19s 316us/step - loss: 0.1802


In [20]:
masked_preds = autoencoder.predict(x_test_masked)
for i in range(10):
    display_png(array_to_img(x_test[i]))
    display_png(array_to_img(x_test_masked[i]))
    display_png(array_to_img(masked_preds[i]))
    print('-'*25)

-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------
